In [ ]:
from flask import *  
import numpy as np
import cv2
from keras.models import load_model
import os
from random import shuffle
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import datasets,layers,models

app = Flask(__name__)  
model=load_model('mymodel.h5')
          
class Predict():
    def test(self,img_path):
        model=load_model('mymodel.h5')
        def my_label(image_name):
            name=image_name.split('.')[0]
            if name in classes:
                pass
            else:
                classes.append(name)
        
        def predict_an_image(img):
            img=np.expand_dims(img,axis=0)
            r=model.predict(img)
            print(r)
            if np.amax(r)<0.5:
                return "unknown"
            return classes[np.argmax(r)]
        
        
        face_classifier=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            faces=face_classifier.detectMultiScale(gray,1.3,5)
            if faces is not ():
                for x,y,w,h in faces:
                    cropped_face=img[y:y+h,x:x+w]
            else:
                return None
            return cropped_face
        
        classes=[]
        for img in tqdm(os.listdir("data")):
            my_label(img)
        
        img=plt.imread(img_path)
        img_data=face_cropped(img)
        if img_data is not None:
            img_data=cv2.cvtColor(img_data,cv2.COLOR_BGR2GRAY)
            img_data=cv2.resize(img_data,(50,50))
            img_data=img_data/255.0
            img_data=img_data.reshape(50,50,1)
            plt.imshow(img_data)
            print(classes)
            return predict_an_image(img_data)
        else:
            return "No faces identified"
#         return "done"
    
@app.route('/')  
def upload():  
    return render_template("home.html")  
 
@app.route('/x', methods = ['POST'])  
def x():  
    if request.method == 'POST':  
        f = request.files['file']
        f.save(f.filename) 
        print(f)
        predict=Predict()
        for dirname, _, filenames in os.walk('/kaggle/input'):
            for filename in filenames:
                print(os.path.join(dirname, filename))
        name=predict.test(f.filename)
        return render_template("x.html", name = name,path = f.filename) 

@app.route('/y', methods = ['POST'])  
def y():  
    if request.method == 'POST':
        
        classes=[]
        def my_label(image_name):
            name=image_name.split('.')[0]
            if name in classes:
                return classes.index(name)
            else:
                classes.append(name)
                return classes.index(name)
        
        face_classifier=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            faces=face_classifier.detectMultiScale(img,1.3,5)
            if faces is not ():
                for x,y,w,h in faces:
                    cropped_face=img[y:y+h,x:x+w]
            else:
                return None
            return cropped_face
        
        def my_data():
            data=[]
            for img in tqdm(os.listdir("data")):
                path=os.path.join("data",img)
                img_data=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                img_data=cv2.resize(img_data,(50,50))
                data.append([np.array(img_data),my_label(img)])
                shuffle(data)
            return data
        
        f1 = request.files['file1']
        nname = request.form['name']
        extension=f1.filename.split('.')[-1]
        f1.save(f1.filename)
        img=plt.imread(f1.filename)
        
        face=cv2.resize(face_cropped(img),(200,203))
        face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        for i in range(1,11):
            file_name_path="data/"+nname+"."+str(i)+"."+extension
            cv2.imwrite(file_name_path,face)
        
        
        data=my_data()
        x=np.array([a[0] for a in data]).reshape(-1,50,50,1)
        y=np.array([a[1] for a in data])
        x=x/255.0
        cnn=models.Sequential([
            layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(50,50,1)),
            layers.MaxPooling2D((2,2)),
    
            layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
            layers.MaxPooling2D((2,2)),
    
            layers.Flatten(),
    
            layers.Dense(128,activation='relu'),
    
            layers.Dense(64,activation='relu'),
    
            layers.Dense(len(classes),activation='softmax')
        ])
        cnn.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
        cnn.fit(x,y,epochs=10)
        cnn.save('mymodel.h5')
        return render_template("home.html") 

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

<>:38: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:96: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:38: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:96: SyntaxWarning: "is not" with a literal. Did you mean "!="?


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


<ipython-input-2-8b403a85f6b6>:38: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if faces is not ():
<ipython-input-2-8b403a85f6b6>:96: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if faces is not ():
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2021 12:18:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 12:18:12] "GET /pic.jpg HTTP/1.1" 404 -


<FileStorage: 'cruise.jpg' ('image/jpeg')>


100%|██████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<?, ?it/s]


...

127.0.0.1 - - [12/Jun/2021 12:18:22] "POST /x HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 12:18:22] "GET /pic.jpg HTTP/1.1" 404 -


[[1.1913192e-01 2.0051652e-03 2.2222632e-01 3.3123660e-04 9.4033097e-04
  6.5536499e-01]]


100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [00:02<00:00, 30.27it/s]


Epoch 1/10
3/3 [==============================] - 3s 92ms/step - loss: 1.9313 - accuracy: 0.1571
Epoch 2/10
3/3 [==============================] - 0s 94ms/step - loss: 1.8559 - accuracy: 0.2000
Epoch 3/10
3/3 [==============================] - 0s 92ms/step - loss: 1.6368 - accuracy: 0.6000
Epoch 4/10
3/3 [==============================] - 0s 98ms/step - loss: 1.3030 - accuracy: 0.7857
Epoch 5/10
3/3 [==============================] - 0s 94ms/step - loss: 0.9495 - accuracy: 0.7429
Epoch 6/10
3/3 [==============================] - 0s 96ms/step - loss: 0.7037 - accuracy: 0.7571
Epoch 7/10
3/3 [==============================] - 0s 115ms/step - loss: 0.5108 - accuracy: 0.7857
Epoch 8/10
3/3 [==============================] - 0s 114ms/step - loss: 0.3947 - accuracy: 0.8857
Epoch 9/10
3/3 [==============================] - 0s 109ms/step - loss: 0.2209 - accuracy: 0.9286
Epoch 10/10
3/3 [==============================] - 0s 117ms/step - loss: 0.2314 - accuracy: 0.9000


127.0.0.1 - - [12/Jun/2021 12:18:48] "POST /y HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2021 12:18:48] "GET /pic.jpg HTTP/1.1" 404 -
